# Thermal Comfort Modeling

**Practical Application: Multi-variable Fuzzy System**

In [ ]:
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fuzzy_systems.core import LinguisticVariable

plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

### 1. Temperature Variable

In [ ]:
# Model temperature: cold (0-20°C), warm (15-29°C), hot (26-40°C)
temperature = LinguisticVariable("temperature", universe=(0, 40))
temperature.add_term("cold", "trapezoidal", (0, 0, 10, 20))
temperature.add_term("warm", "triangular", (15, 22, 29))
temperature.add_term("hot", "trapezoidal", (26, 32, 40, 40))

temperature.plot(figsize=(12, 5), show=True)

### 2. Humidity Variable

In [ ]:
# Model humidity: low (0-40%), normal (30-70%), high (60-100%)
humidity = LinguisticVariable("humidity", universe=(0, 100))
humidity.add_term("low", "trapezoidal", (0, 0, 20, 40))
humidity.add_term("normal", "triangular", (30, 50, 70))
humidity.add_term("high", "trapezoidal", (60, 80, 100, 100))

humidity.plot(figsize=(12, 5), show=True)

In [ ]:
# Side-by-side visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Temperature
temp_x = np.linspace(0, 40, 500)
colors_temp = {'cold': 'b', 'warm': 'g', 'hot': 'r'}
for name, term in temperature.terms.items():
    mu = term.membership(temp_x)
    ax1.plot(temp_x, mu, color=colors_temp[name], linewidth=2, label=name.upper())
    ax1.fill_between(temp_x, 0, mu, alpha=0.2, color=colors_temp[name])
ax1.set_xlabel('Temperature (°C)')
ax1.set_ylabel('μ(T)')
ax1.set_title('TEMPERATURE')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Humidity
hum_x = np.linspace(0, 100, 500)
colors_hum = {'low': 'b', 'normal': 'g', 'high': 'r'}
for name, term in humidity.terms.items():
    mu = term.membership(hum_x)
    ax2.plot(hum_x, mu, color=colors_hum[name], linewidth=2, label=name.upper())
    ax2.fill_between(hum_x, 0, mu, alpha=0.2, color=colors_hum[name])
ax2.set_xlabel('Humidity (%)')
ax2.set_ylabel('μ(H)')
ax2.set_title('HUMIDITY')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3. Fuzzy Rules for Comfort

In [ ]:
def calculate_comfort(temp_val, hum_val):
    """
    Calculate thermal comfort using fuzzy rules.
    
    Rules:
    - Ideal comfort: warm AND normal
    - Good comfort: (warm AND low) OR (cold AND normal)
    - Discomfort: (hot AND high) OR (cold AND high) OR (hot AND low)
    """
    mu_temp = temperature.fuzzify(temp_val)
    mu_hum = humidity.fuzzify(hum_val)
    
    # Apply fuzzy rules using min (AND) and max (OR)
    ideal = min(mu_temp['warm'], mu_hum['normal'])
    good = max(
        min(mu_temp['warm'], mu_hum['low']),
        min(mu_temp['cold'], mu_hum['normal'])
    )
    discomfort = max(
        max(
            min(mu_temp['hot'], mu_hum['high']),
            min(mu_temp['cold'], mu_hum['high'])
        ),
        min(mu_temp['hot'], mu_hum['low'])
    )
    
    return mu_temp, mu_hum, ideal, good, discomfort

### 4. Test Cases

In [ ]:
# Test different conditions
test_cases = [
    (22, 50, "Ideal conditions"),
    (18, 45, "Cool and dry"),
    (32, 75, "Hot and humid"),
    (25, 30, "Good temp, dry air"),
    (28, 65, "Slightly warm and humid")
]

for temp, hum, desc in test_cases:
    mu_t, mu_h, ideal, good, disc = calculate_comfort(temp, hum)
    print(f"\n{desc}: T={temp}°C, H={hum}%")
    print(f"  Temperature: cold={mu_t['cold']:.2f}, warm={mu_t['warm']:.2f}, hot={mu_t['hot']:.2f}")
    print(f"  Humidity: low={mu_h['low']:.2f}, normal={mu_h['normal']:.2f}, high={mu_h['high']:.2f}")
    print(f"  Comfort: ideal={ideal:.2f}, good={good:.2f}, discomfort={disc:.2f}")

### 5. Thermal Comfort Map

In [ ]:
# Create 2D comfort map for all temperature-humidity combinations
temps = np.linspace(0, 40, 100)
hums = np.linspace(0, 100, 100)
comfort_map = np.zeros((len(hums), len(temps)))

for i, h in enumerate(hums):
    for j, t in enumerate(temps):
        _, _, ideal, good, _ = calculate_comfort(t, h)
        comfort_map[i, j] = max(ideal, 0.5 * good)  # Weighted comfort score

plt.figure(figsize=(12, 8))
contour = plt.contourf(temps, hums, comfort_map, levels=20, cmap='RdYlGn')
plt.colorbar(contour, label='Comfort Degree')
plt.xlabel('Temperature (°C)')
plt.ylabel('Humidity (%)')
plt.title('Thermal Comfort Map (Green = More Comfortable)')
plt.grid(True, alpha=0.3, color='white')

# Mark ideal region
plt.plot([15, 29, 29, 15, 15], [30, 30, 70, 70, 30], 'b--', linewidth=2, label='Ideal range')
plt.legend()
plt.show()

### 🎯 Exercise 1: Test Your Own Values

In [ ]:
# Modify these values to test different conditions
my_temperature = 25  # °C
my_humidity = 60     # %

mu_t, mu_h, ideal, good, disc = calculate_comfort(my_temperature, my_humidity)
print(f"Your test: T={my_temperature}°C, H={my_humidity}%")
print(f"  Comfort scores: ideal={ideal:.3f}, good={good:.3f}, discomfort={disc:.3f}")

### 🎯 Exercise 2: Modify Membership Functions

In [ ]:
# Create custom temperature model with different parameters
temp_custom = LinguisticVariable("temperature_custom", universe=(0, 40))
temp_custom.add_term("cold", "trapezoidal", (0, 0, 12, 18))   # Try different values
temp_custom.add_term("warm", "triangular", (16, 24, 28))
temp_custom.add_term("hot", "trapezoidal", (25, 30, 40, 40))

temp_custom.plot(figsize=(12, 5), show=True)

### 🎯 Exercise 3: Add New Rules

In [ ]:
# Implement your own comfort rules
def custom_comfort_rules(temp_val, hum_val):
    mu_temp = temperature.fuzzify(temp_val)
    mu_hum = humidity.fuzzify(hum_val)
    
    # TODO: Add your own rules here
    # Example:
    # rule1 = min(mu_temp['cold'], mu_hum['low'])
    # rule2 = min(mu_temp['hot'], mu_hum['normal'])
    
    pass

# Test your custom rules